In [34]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import seaborn as sns


##forcast solar energy generation
#put into optimiser -> battery schedule
#do exploratory analysis
#dic of vars(nas, pattern of na, odd values, errors, avg generation value[all generator])
#assume that audience has 1st year level -> doc through all the working process
#build model can use external factors to predict
#ex: temp forcast to foracst solar energy generation -> prototype
#goals: building models and evaluate all models that you've built(dif model can apply for dif vars->experiment and combine)
#keep track all models (how far predicted values from actual values)
#finalising report and presentation
#docs all info and keep clean github
#time series data
#use phase 1+2 as training(forecast)
#test data to test model
#tuning parameters
#multiverate, correlation

In [35]:
temp = pd.read_csv('Solar_Data.csv')#reading temp dataset

In [36]:
temp_date = pd.DataFrame({
    'year': temp['Year'],
    'month':temp['Month'],
    'day': temp["Day"]}
)
date_time = pd.to_datetime(temp_date[['year','month','day']])
temp['date_temp']= date_time

In [37]:
temp= temp.drop(['Year', 'Month','Day'], axis=1)
temp

,Product code,Bureau of Meteorology station number,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality,date_temp
0,IDCJAC0010,86338,NaN,NaN,NaN,2013-01-01
1,IDCJAC0010,86338,NaN,NaN,NaN,2013-01-02
2,IDCJAC0010,86338,NaN,NaN,NaN,2013-01-03
3,IDCJAC0010,86338,NaN,NaN,NaN,2013-01-04
4,IDCJAC0010,86338,NaN,NaN,NaN,2013-01-05
...,...,...,...,...,...,...
3518,IDCJAC0010,86338,13.7,1.0,N,2022-08-20
3519,IDCJAC0010,86338,15.1,1.0,N,2022-08-21
3520,IDCJAC0010,86338,18.1,1.0,N,2022-08-22
3521,IDCJAC0010,86338,13.1,1.0,N,2022-08-23


In [38]:
temp=temp.dropna()#dropping na

In [39]:
temp.head(40)

,Product code,Bureau of Meteorology station number,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality,date_temp
151,IDCJAC0010,86338,15.8,1.0,Y,2013-06-01
152,IDCJAC0010,86338,15.7,1.0,Y,2013-06-02
153,IDCJAC0010,86338,14.8,1.0,Y,2013-06-03
154,IDCJAC0010,86338,15.0,1.0,Y,2013-06-04
155,IDCJAC0010,86338,14.6,1.0,Y,2013-06-05
156,IDCJAC0010,86338,17.3,1.0,Y,2013-06-06
157,IDCJAC0010,86338,14.5,1.0,Y,2013-06-07
158,IDCJAC0010,86338,15.7,1.0,Y,2013-06-08
159,IDCJAC0010,86338,14.4,1.0,Y,2013-06-09
160,IDCJAC0010,86338,16.9,1.0,Y,2013-06-10


In [14]:
temp['Days of accumulation of maximum temperature'].unique

<bound method Series.unique of 151     1.0
152     1.0
153     1.0
154     1.0
155     1.0
       ... 
3518    1.0
3519    1.0
3520    1.0
3521    1.0
3522    1.0
Name: Days of accumulation of maximum temperature, Length: 3369, dtype: float64>

In [42]:
temp = temp.set_index('date_temp').sort_index()
temp_phase1 = temp[pd.Timestamp("2016-07-03"):pd.Timestamp("2020-09-30")]


In [43]:
temp_phase1

,Product code,Bureau of Meteorology station number,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality
date_temp,,,,,
2016-07-03,IDCJAC0010,86338,16.7,1.0,Y
2016-07-04,IDCJAC0010,86338,12.7,1.0,Y
2016-07-05,IDCJAC0010,86338,12.8,1.0,Y
2016-07-06,IDCJAC0010,86338,13.0,1.0,Y
2016-07-07,IDCJAC0010,86338,13.7,1.0,Y
...,...,...,...,...,...
2020-09-26,IDCJAC0010,86338,13.4,1.0,Y
2020-09-27,IDCJAC0010,86338,13.4,1.0,Y
2020-09-28,IDCJAC0010,86338,13.6,1.0,Y
